In [3]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
from keras.layers import Convolution1D, MaxPooling1D
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [52]:
print("Data read started...")
data = pd.read_csv("result1.csv")
data = data.as_matrix()
print ("Data read finished.")

touch_only = 'false'
touch_heart = 'false'
print(data.shape)

Data read started...
Data read finished.
(286, 15)


# Eliminate EEG data

In [53]:
data = np.delete(data, 13, 1)    # TouchNumber
data = np.delete(data, 12, 1)    # T/S
data = np.delete(data, 11, 1)    # TouchTimestamp
data = np.delete(data, 4, 1)     # RRInterval
data = np.delete(data, 3, 1)     # hpvPower
data = np.delete(data, 2, 1)     # Heartrate
data = np.delete(data, 1, 1)     # Timestamp
data.shape

if(touch_only == 'true'):
    data = np.delete(data, 7, 1)     # SDNN
    data = np.delete(data, 3, 1)     # Burg
    data = np.delete(data, 2, 1)     # ReactionTime

if(touch_heart == 'true'):
    data = np.delete(data, 7, 1)     # ReactionTime
    
data.shape

(286, 8)

In [54]:
one = 0
two = 0
three = 0

for i in range(len(data)):
    if (data[i, 0] == 'NbackSym1'):
        one = one + 1
    if (data[i, 0] == 'NbackSym2'):
        two = two + 1
    if (data[i, 0] == 'NbackSym3'):
        three = three + 1

print(' one: ', one, ' two: ', two, ' three: ', three)

 one:  81  two:  105  three:  100


# Dictionary for the levels

In [55]:
level=["NbackSym1","NbackSym2","NbackSym3"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [56]:
for i in range (1,data.shape[1]):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [57]:
back1 = np.zeros((one,data.shape[1]-1))
back2 = np.zeros((two,data.shape[1]-1))
back3 = np.zeros((three,data.shape[1]-1))
j = 0
k = 0
l = 0

for i in range(len(data)):
    if (data[i, 0] == 'NbackSym1'):
        back1[j] = data[i, 1:]
        j = j+1
    if (data[i, 0] == 'NbackSym2'):
        back2[k] = data[i, 1:]
        k = k+1
    if (data[i, 0] == 'NbackSym3'):
        back3[l] = data[i, 1:]
        l = l+1

In [58]:
y_data = data[:, 0]
for i in range(len(data)):
    y_data[i] = level2int[y_data[i]]

# One-hot encoding

In [59]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()

# Group data for convolutional network

In [60]:
track = 20
null = np.array([0,0,0,0,0,0,0])

for i in range(track-1):
        back1 = np.vstack([back1, null])
        back2 = np.vstack([back2, null])
        back3 = np.vstack([back3, null])

In [61]:
x_data = np.zeros((data.shape[0],back1.shape[1],track))         # final input data for the network
y_one_hot = np.zeros((data.shape[0],one_hot.shape[1]))          # to store one-hot data groupped

for i in range(len(one_hot)):
    y_one_hot[i]=one_hot[i]

for i in range(len(back1)-track+1):
    for j in range(track-1):
        x_data[i, :, j] = back1[i+j]
        
index = 0
for i in range(len(back1)-track+1, len(back1)-track+1 + len(back2)-track+1):
    for j in range(track-1):
        x_data[i, :, j] = back2[index+j]
    index = index+1
    
index = 0
for i in range(len(back1)-track+1 + len(back2)-track+1, len(back1)-track+1 + len(back2)-track+1 + len(back3)-track+1):
    for j in range(track-1):
        x_data[i, :, j] = back3[index+j]
    index = index+1

# Shuffle data

In [62]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [63]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [64]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(200, 7, 20) (200, 3) (42, 7, 20) (42, 3) (44, 7, 20) (44, 3)


# Build the net

In [65]:
model = Sequential()

model.add(Convolution1D(input_shape=(x_train[0].shape[-2],x_train[0].shape[-1]),
                        nb_filter=35,
                        filter_length=4,
                        border_mode='same',
                        subsample_length=2,
                        init='glorot_normal',
                        activation='relu')) 

#model.add(MaxPooling1D(pool_length=2,stride=1))
model.add(Dropout(0.5))

model.add(Convolution1D(nb_filter=30,
                        filter_length=2,
                        border_mode='same',
                        subsample_length=2,
                        init='glorot_normal',
                        activation='relu'))

model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(y_train.shape[1], activation='softmax'))

In [66]:
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])


model.fit(x_train, y_train, nb_epoch=200, shuffle=True, callbacks=[earlyStopping], 
          validation_data = (x_valid, y_valid), batch_size=50)

Train on 200 samples, validate on 42 samples
Epoch 1/200
200/200 [==============================] - 0s - loss: 1.0923 - acc: 0.4200 - val_loss: 1.0676 - val_acc: 0.6905
Epoch 2/200
200/200 [==============================] - 0s - loss: 1.0632 - acc: 0.5950 - val_loss: 1.0281 - val_acc: 0.8095
Epoch 3/200
200/200 [==============================] - 0s - loss: 1.0151 - acc: 0.6800 - val_loss: 0.9724 - val_acc: 0.8810
Epoch 4/200
200/200 [==============================] - 0s - loss: 0.9587 - acc: 0.7450 - val_loss: 0.8970 - val_acc: 0.8810
Epoch 5/200
200/200 [==============================] - 0s - loss: 0.8872 - acc: 0.7500 - val_loss: 0.8028 - val_acc: 0.9048
Epoch 6/200
200/200 [==============================] - 0s - loss: 0.8086 - acc: 0.8050 - val_loss: 0.7007 - val_acc: 0.9048
Epoch 7/200
200/200 [==============================] - 0s - loss: 0.7362 - acc: 0.7950 - val_loss: 0.6012 - val_acc: 0.9286
Epoch 8/200
200/200 [==============================] - 0s - loss: 0.6366 - acc: 0.8500 

In [67]:
model.evaluate(x_test, y_test, batch_size=50)

44/44 [==============================] - 0s


[0.09788048267364502, 0.93181818723678589]

In [68]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(0back)', 'class 1(1back)', 'class 2(2back)', 'class3(3back)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

44/44 [==============================] - 0s     
[2 2 0 2 1 0 0 0 2 1 2 2 1 0 2 0 0 2 1 0 0 0 1 2 0 2 0 2 2 0 1 2 1 2 1 2 0
 0 2 1 2 1 2 2]
32/44 [====================>.........] - ETA: 0s                precision    recall  f1-score   support

class 0(0back)       1.00      1.00      1.00        15
class 1(1back)       0.90      0.82      0.86        11
class 2(2back)       0.89      0.94      0.92        18

   avg / total       0.93      0.93      0.93        44

[[15  0  0]
 [ 0  9  2]
 [ 0  1 17]]
